In [56]:
from conf import download_path, db_conn, kite as kite_conf
import polars as pl
from src.scans.swing_scan import basic_filter, add_basic_indicators
from datetime import datetime
from src.utils import setup_logger
import logging

setup_logger()

logger = logging.getLogger(__name__)

In [57]:
END_DATE = datetime(2025, 12, 1)

In [58]:
scan_symbol_list = (
    pl.scan_parquet(download_path / "adr_stocks.parquet")
    .collect()
    .get_column("symbol")
    .to_list()
)

logger.info(f"Stocks in the Scan List {len(scan_symbol_list)}")

query = f"""
select * 
from {kite_conf["hist_table_name"]}
where symbol in {tuple(scan_symbol_list)}
"""
data = pl.read_database_uri(query=query, uri=db_conn)

basic_stock_list = basic_filter(
    data=data, symbol_list=scan_symbol_list, scan_date=END_DATE
)
query = f"""
select * 
from {kite_conf["hist_table_name"]}
where symbol in {tuple(basic_stock_list)}
"""
data = pl.read_database_uri(query=query, uri=db_conn)

2025-12-14 00:56:59 | INFO | __main__ | <module> | Stocks in the Scan List 498
2025-12-14 00:56:59 | INFO | src.scans.swing_scan | basic_filter | Number of stocks in symbol list: 498
2025-12-14 00:56:59 | INFO | src.scans.swing_scan | basic_filter | Symbols after basic filter: 207


# Reddit Comment

In [59]:
# WINDOW_SIZE = 10

# (
#     data.lazy()
#     .with_columns(
#         # Shift Columns
#         [
#             pl.col(col)
#             .shift(i)
#             .over(partition_by="symbol", order_by="timestamp", descending=False)
#             .alias(f"{col}_prev_{i}")
#             for col in ["close", "timestamp"]
#             for i in [1]
#         ]
#     )
#     # Calculate Ranges
#     .with_columns(
#         (
#             (pl.max_horizontal(pl.col("close") - pl.col("close_prev_1")))
#             - pl.min_horizontal(pl.col("low") - pl.col("close_prev_1"))
#         ).alias("small_range"),
#         (
#             pl.col("close").rolling_max(window_size=WINDOW_SIZE)
#             - pl.col("low").rolling_min(window_size=WINDOW_SIZE)
#         )
#         .over(partition_by="symbol", order_by="timestamp", descending=False)
#         .alias("large_range"),
#     )
#     .with_columns(
#         (
#             (
#                 pl.col("small_range").rolling_sum(window_size=WINDOW_SIZE)
#                 / pl.col("large_range")
#             ).log()
#             / pl.lit(10).log()
#         )
#         .over(partition_by="symbol", order_by="timestamp", descending=False)
#         .alias("R")
#     )
#     .with_columns(
#         ((pl.col("R") > 0.6).cast(pl.Int64).rolling_sum(window_size=5) >= 5)
#         .over(partition_by="symbol", order_by="timestamp", descending=False)
#         .alias("scan")
#     )
#     .filter(pl.col("scan") == True)
#     .filter(pl.col("timestamp") == datetime(2025, 9, 4))
# ).collect()

# PullBack Exp

In [67]:
NEAR_PCT = 0.01

comparisons = [
    (pl.col(f"mid_prev_{i}")) <= pl.col(f"mid_prev_{i + 1}") for i in range(1, 10)
]
df = add_basic_indicators(data=data)
pullback_filter = (
    df.lazy()
    .with_columns([pl.mean_horizontal(("open", "close")).round(2).alias("mid_prev_0")])
    .with_columns(
        [
            pl.col("mid_prev_0")
            .shift(i)
            .over(partition_by="symbol", order_by="timestamp", descending=False)
            .alias(f"mid_prev_{i}")
            for i in range(1, 11)
        ]
        + [
            (
                ((pl.col("mid_prev_0") - pl.col(col)).abs() / pl.col(col)) <= NEAR_PCT
            ).alias(f"near_{col}")
            for col in ["close_ema_9", "close_ema_21", "close_sma_50"]
        ]
    )
    .with_columns(
        pl.sum_horizontal(
            [pl.when(cond).then(1).otherwise(0) for cond in comparisons]
        ).alias("mid_down_count")
    )
    .filter(
        (
            (pl.col("near_close_ema_9") == True)
            | (pl.col("near_close_ema_21") == True)
            | (pl.col("near_close_sma_50") == True)
        )
        & (pl.col("mid_down_count") > 2)
        & (pl.col("timestamp") == END_DATE)
        & (pl.col("adr_pct_20") >= 3.5)
        & (pl.col("rvol_pct") < 50)
    )
    .sort(["rvol_pct", "adr_pct_20"], descending=[False, True])
    .with_row_index(name="rank", offset=1)
).collect()

In [68]:
pullback_filter

rank,symbol,timestamp,open,high,low,close,volume,close_sma_50,close_ema_9,close_ema_21,volume_sma_20,volume_sma_50,day_range,adr_pct_20,rvol_pct,mid_prev_0,mid_prev_1,mid_prev_2,mid_prev_3,mid_prev_4,mid_prev_5,mid_prev_6,mid_prev_7,mid_prev_8,mid_prev_9,mid_prev_10,near_close_ema_9,near_close_ema_21,near_close_sma_50,mid_down_count
u32,str,date,f64,f64,f64,f64,i64,f64,f64,f64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,bool,bool,bool,i32
1,"""RAMAPHO""",2025-12-01,175.52,180.38,172.1,172.7,66391,174.47,178.59,181.71,380870,559305,1.048112,5.81,12.0,174.11,177.83,177.14,176.28,174.74,177.39,184.52,183.98,183.48,186.15,188.62,false,false,true,4
2,"""BAJAJCON""",2025-12-01,267.4,268.25,262.4,265.7,165096,264.87,268.22,270.77,963417,1101554,1.022294,3.91,15.0,266.55,266.12,265.85,264.5,263.58,264.23,272.15,277.23,277.05,281.98,275.77,true,false,true,4
3,"""THYROCARE""",2025-12-01,489.0,492.0,482.5,485.0,292805,443.02,494.94,484.24,1256957,1705442,1.019689,4.87,17.0,487.0,502.85,495.9,501.05,494.7,502.4,511.0,515.5,525.95,527.5,526.3,false,true,false,6
4,"""RAJRATAN""",2025-12-01,433.0,442.0,426.5,438.95,90135,390.4,440.3,436.51,448627,441433,1.036342,5.43,20.0,435.98,434.95,432.12,439.05,430.85,429.85,442.8,448.55,463.72,484.22,484.15,true,true,false,5
5,"""PGIL""",2025-12-01,1688.7,1699.0,1666.0,1671.1,49783,1428.81,1688.78,1610.27,515029,242006,1.019808,5.68,21.0,1679.9,1694.35,1731.15,1735.1,1718.05,1731.5,1751.55,1764.85,1722.45,1676.5,1673.5,true,false,false,5
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
39,"""ARVIND""",2025-12-01,352.95,354.9,341.8,345.45,221778,320.9,350.79,341.93,655224,460095,1.038327,3.75,48.0,349.2,354.0,358.48,358.48,359.02,353.75,351.45,358.75,354.72,341.18,336.48,true,false,false,4
40,"""PRECWIRE""",2025-12-01,257.2,264.2,244.0,246.0,624620,219.96,251.62,244.4,1795034,1282662,1.082787,5.58,49.0,251.6,257.01,255.92,251.79,245.71,251.42,258.06,257.3,256.26,261.43,269.42,true,false,false,4
41,"""SPANDANA""",2025-12-01,265.0,267.35,261.7,262.75,180002,255.72,266.19,262.15,675021,365559,1.02159,5.53,49.0,263.88,265.52,268.95,271.92,268.18,265.45,271.73,276.82,279.58,276.32,274.7,true,true,false,5


# AMIBROKER

In [62]:
data.select(pl.col("symbol").n_unique())

symbol
u32
207


In [63]:
timeframe = 252
vol_tf = 50
base_lower_limit = 0.6
pivot_length = 5
pv_limit = 0.1

ami_scan_stocks = (
    (
        data.lazy()
        .with_columns(
            # 52 week high calculation
            pl.col("close")
            .rolling_max(window_size=timeframe)
            .over(partition_by="symbol", order_by="timestamp", descending=False)
            .alias("52_week_high"),
            # volume sma calculation
            pl.col("volume")
            .rolling_mean(window_size=vol_tf)
            .over(partition_by="symbol", order_by="timestamp", descending=False)
            .alias(f"volume_sma_{vol_tf}"),
            # pivot high calculation
            pl.col("high")
            .rolling_max(window_size=pivot_length)
            .over(partition_by="symbol", order_by="timestamp", descending=False)
            .alias("pivot_high"),
            # pivot low calculation
            pl.col("low")
            .rolling_min(window_size=pivot_length)
            .over(partition_by="symbol", order_by="timestamp", descending=False)
            .alias("pivot_low"),
            # pivot start high
            pl.col("high")
            .shift(pivot_length - 1)
            .over(partition_by="symbol", order_by="timestamp", descending=False)
            .alias("pivot_start_high"),
        )
        .with_columns(
            # pivot width
            ((pl.col("pivot_high") - pl.col("pivot_low")) / pl.col("close")).alias(
                "pivot_width"
            )
        )
        .with_columns(
            # find pivot
            (
                (pl.col("pivot_width") < pv_limit)
                & (pl.col("pivot_high") == pl.col("pivot_start_high"))
            ).alias("is_pivot"),
            # volume dry up
            pl.all_horizontal(
                [
                    pl.col("volume").shift(i) < pl.col(f"volume_sma_{vol_tf}").shift(i)
                    for i in range(pivot_length)
                ]
            )
            .over(partition_by="symbol", order_by="timestamp", descending=False)
            .alias("vol_dry_up"),
            # near 52 week high
            (
                (pl.col("close") < pl.col("52_week_high"))
                & (pl.col("close") > base_lower_limit * pl.col("52_week_high"))
            ).alias("near_high"),
        )
        .filter(
            pl.col("near_high") & pl.col("is_pivot") & pl.col("vol_dry_up")
            # & pl.col("vol_decreasing")
        )
    )
    .collect()
    .filter(pl.col("timestamp") == END_DATE)
    .sort("symbol")
    # .get_column("symbol")
    # .to_list()
)

# ami_final_stocks = basic_filter(data=data, symbol_list=ami_scan_stocks, scan_date=END_DATE)
# ami_final_stocks

In [64]:
ami_scan_stocks

symbol,timestamp,open,high,low,close,volume,52_week_high,volume_sma_50,pivot_high,pivot_low,pivot_start_high,pivot_width,is_pivot,vol_dry_up,near_high
str,datetime[ns],f64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,bool,bool,bool
"""AETHER""",2025-12-01 00:00:00,875.0,909.0,875.0,902.35,313236,932.95,322385.2,923.0,844.7,923.0,0.086773,true,true,true
"""AGIIL""",2025-12-01 00:00:00,273.4,282.55,267.15,271.0,485615,287.9,1.1810e6,289.0,267.15,289.0,0.080627,true,true,true
"""GMMPFAUDLR""",2025-12-01 00:00:00,1124.0,1137.5,1101.1,1109.2,44876,1403.0,189791.76,1176.2,1101.1,1176.2,0.067706,true,true,true
"""GRMOVER""",2025-12-01 00:00:00,467.1,478.15,453.65,474.3,393257,494.35,805572.36,479.8,453.35,479.8,0.055766,true,true,true
"""INDIAGLYCO""",2025-12-01 00:00:00,1094.0,1099.1,1069.1,1081.0,119881,1155.8,355491.06,1139.95,1069.1,1139.95,0.065541,true,true,true
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""PRICOLLTD""",2025-12-01 00:00:00,624.05,630.0,619.1,621.95,193740,661.1,899119.1,641.45,614.0,641.45,0.044135,true,true,true
"""REDINGTON""",2025-12-01 00:00:00,281.0,287.5,280.65,282.15,2103821,329.7,8.5641e6,292.0,277.55,292.0,0.051214,true,true,true
"""SAKSOFT""",2025-12-01 00:00:00,204.65,205.5,201.52,203.3,93535,247.98,293917.98,211.0,196.65,211.0,0.070585,true,true,true
